In [3]:
import csv
import os.path
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from rbflayer import RBFLayer, InitCentersRandom
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import matplotlib.pyplot as plt
import math

In [4]:
# http://benalexkeen.com/feature-scaling-with-scikit-learn/
# https://github.com/PetraVidnerova/rbf_keras/blob/master/rbflayer.py

def load_dataset(is_normalize=True):
    global dataset, labels
    with open('OnlineNewsPopularity.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        next(reader) # skip the first row
        temp = []
        for idx, row in enumerate(reader):
            temp.append(row[1:])
        if is_normalize:
            # it is important which type of normalization method you use
            scaler = preprocessing.MinMaxScaler()
            temp = scaler.fit_transform(temp)
            print('Data is normalize')
        else:
            print('No normalize apply')
        for idx, row in enumerate(temp):
            slice_row = np.append(row[1:45], row[46:])
            labels.append(row[45])
            dataset.append(slice_row)
    return np.array(dataset), np.array(labels).reshape(len(labels), 1)

In [5]:
# Load dataset 
dataset = []
labels = []
# ori_dataset, ori_labels = load_dataset(is_normalize=False) 
dataset, labels = load_dataset(is_normalize=True)
print('Dataset shape: {0}, Labels: {1}'.format(dataset.shape, labels.shape))

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)
print("x_train: {0}, y_train: {1}".format(x_train.shape, y_train.shape))
print("x_test: {0}, y_test: {1}".format(x_test.shape, y_test.shape))

Data is normalize
Dataset shape: (39644, 58), Labels: (39644, 1)
x_train: (31715, 58), y_train: (31715, 1)
x_test: (7929, 58), y_test: (7929, 1)


In [6]:
print(x_train[0])
print(y_train[:20])

[6.19047619e-01 7.41090394e-02 7.27116601e-04 9.59692896e-04
 9.92436817e-04 4.60526316e-02 8.62068966e-03 7.81250000e-03
 0.00000000e+00 6.33058894e-01 6.66666667e-01 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.26876676e-03 7.03994636e-03
 9.01221392e-03 1.00000000e+00 3.95842862e-01 4.23732082e-01
 2.12057641e-02 8.72852433e-02 8.73947587e-04 8.73947587e-04
 8.73947587e-04 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.08539436e-01 3.08572373e-02 3.13792118e-02
 4.77560112e-01 3.09215853e-02 3.16003500e-01 3.93611889e-01
 6.02736494e-02 6.95652174e-01 3.04347826e-01 2.75970644e-01
 3.33333333e-02 8.00000000e-01 7.40816327e-01 5.00000000e-01
 9.00000000e-01 4.58333333e-01 6.66666667e-01 8.33333333e-02
 3.33333333e-01 1.90905005e-02]
[[0.16385663]
 [0.29876792]
 [0.31690821]
 [0.44456492]
 [0.26143791]
 [0.44870041]
 [0.28425956]
 [0.23037752]
 [

In [2]:
# https://www.youtube.com/watch?v=1Cw45yNm6VA
# build model
# beta - controls the drop off of circle. higher the beta the faster the circle will drop off. This will then affect the 
#        size of the circle, faster drop off will cause new circle to form.
#        (Drop of refers of gradient of circle, how smooth transit from circle area to non-circle area)
model = Sequential()
rbflayer = RBFLayer(58,
                    initializer=InitCentersRandom(x_train), 
                    betas=1.5,
                    input_shape=(58,))
model.add(rbflayer)
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer=RMSprop(lr=0.001),
              metrics=['mse'])

model.summary()

NameError: name 'x_train' is not defined

In [ ]:
# training with callbacks
saved_weights_name = 'best_weights_rbf.h5'

# load weights if any
if os.path.isfile(saved_weights_name):
    model.load_weights(saved_weights_name)
    print("Weights loaded: {0}".format(saved_weights_name))

early_stop = EarlyStopping(monitor='val_loss',
                           min_delta=0.0001,
                           patience=10,
                           mode='min',
                           verbose=1)

checkpoint = ModelCheckpoint(saved_weights_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min',
                             period=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=4,
                              min_lr=0.00001,
                              verbose=1)
    
model.fit(x_train, y_train,
          epochs=50,
          validation_data=(x_test, y_test),
          callbacks=[early_stop, checkpoint, reduce_lr],
          verbose=1)

In [ ]:
# showing results
scores = model.evaluate(x_test, y_test)
print("{0}: {1}".format(model.metrics_names[0], round(scores[0], 6)))
print("{0}: {1}".format(model.metrics_names[1], round(scores[1], 6)))

# http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
y_pred = model.predict(x_test)
var_score = explained_variance_score(y_pred, y_test)
mean_abs_error = mean_absolute_error(y_pred, y_test)
mean_sqare_log_error = mean_squared_log_error(y_pred, y_test)
median_abs_error = median_absolute_error(y_pred, y_test)
r2 = r2_score(y_pred, y_test)
print('explained_variance_score: {0}'.format(round(var_score, 6)))
print('mean_absolute_error: {0}'.format(round(mean_abs_error, 6)))
print('mean_squared_log_error: {0}'.format(round(mean_sqare_log_error, 6)))
print('median_absolute_error: {0}'.format(round(median_abs_error, 6)))
print('r2_score: {0}'.format(round(r2, 6)))

plt.figure(figsize=(14,6))
plt.plot(y_pred[:100], label="predict")
plt.plot(y_test[:100], label="actual")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.show()